# Dog Breed Classifier

## Project

I always wanted to build a model that could be used by anyone. *'Tell me what type of dog it is'* is a simple idea that could be made into an app. It would tell you the type of dog on a picture or... the one that you resemble. The main problem is to build a classifier strong enough to work not only with clean and prepared data, but with more diverse one like user made photos.

Here I'm going to build a model for it. I will use a convolutional neural network (CNN), a network type doing really well with image recognition. First, I'm going to use pre-trained models to know if it is an image of a dog or a human. Then I will use a CNN to predict the matching breed. You will see me building two CNNs, one from scratch and the other one using transfer learning. I'm curious how they will compare.

## Imports

In [1]:
import numpy as np
from glob import glob

## Load data

The data are images of famous people and different types of dogs.

In [2]:
# load filenames for human and dog images
human_files = np.array(glob('./data/lfw/*/*'))
dog_files = np.array(glob('./data/dog_images/*/*/*'))

# print number of images in each dataset
print('There are %d total human images.' % len(human_files))
print('There are %d total dog images.' % len(dog_files))

There are 13233 total human images.
There are 8351 total dog images.
